In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

2023-12-19 11:38:56.331689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 11:38:56.365480: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 11:38:56.366377: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 11:38:56.932485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Read the dataset
data = pd.read_csv('output_trace.csv')

# Encode categorical columns using LabelEncoder
categorical_columns = ['Source UUID', 'Source type', 'Destination UUID', 'Destination type', 'Edge type']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

In [14]:
# Split the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the target variable (Label)
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
# Create a neural network model
def create_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)


In [18]:
# # Number of clients
# num_clients = 5
# client_models = [create_model() for _ in range(num_clients)]

# for model in client_models:
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)

# # Number of rounds
# num_rounds = 5
# epochs_per_round = 6


# Number of clients
num_clients = 5

# Create and compile each client model independently
client_models = []
for _ in range(num_clients):
    model = create_model()  # Create a new model instance
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use a new optimizer instance
    client_models.append(model)

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)

# Number of rounds
num_rounds = 5
epochs_per_round = 6



In [19]:
# for r in range(num_rounds):
#     print(f"Round {r + 1}/{num_rounds}")
#     for i, model in enumerate(client_models):
#         print(f"Training client {i + 1}/{num_clients}")
#         model.fit(X_train, y_train, epochs=epochs_per_round, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])

#     # Average the clients' model weights
#     avg_weights = [client_models[0].get_weights()]
#     for i in range(1, num_clients):
#         weights = client_models[i].get_weights()
#         for j in range(len(weights)):
#             avg_weights[0][j] += weights[j]
#     for j in range(len(avg_weights[0])):
#         avg_weights[0][j] /= num_clients

#     # Set the new average weights to all client models
#     for model in client_models:
#         model.set_weights(avg_weights[0])
import time

total_training_time = 0

for r in range(num_rounds):
    print(f"Round {r + 1}/{num_rounds}")
    round_training_times = []

    for i, model in enumerate(client_models):
        start_time = time.time()
        print(f"Training client {i + 1}/{num_clients}")
        model.fit(X_train, y_train, epochs=epochs_per_round, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])
        training_time = time.time() - start_time
        round_training_times.append(training_time)

    # Calculate and print average training time for this round
    average_time_this_round = sum(round_training_times) / len(round_training_times)
    print(f"Average training time in round {r + 1}: {average_time_this_round:.2f} seconds")

    total_training_time += sum(round_training_times)

    # Rest of your code for averaging and setting weights

# Calculate and print the total average training time
total_average_training_time = total_training_time / (num_rounds * num_clients)
print(f"Total average training time: {total_average_training_time:.2f} seconds")

Round 1/5
Training client 1/5
Epoch 1/6
22597/22597 [==============================] - 76s 3ms/step - loss: -0.0786 - accuracy: 0.9958 - val_loss: 0.0062 - val_accuracy: 0.9974 - lr: 0.0010
Epoch 2/6
22597/22597 [==============================] - 75s 3ms/step - loss: -0.1717 - accuracy: 0.9969 - val_loss: 0.0065 - val_accuracy: 0.9969 - lr: 0.0010
Epoch 3/6
22597/22597 [==============================] - 75s 3ms/step - loss: -0.3284 - accuracy: 0.9972 - val_loss: 0.0053 - val_accuracy: 0.9978 - lr: 0.0010
Epoch 4/6
22597/22597 [==============================] - 75s 3ms/step - loss: -0.2464 - accuracy: 0.9973 - val_loss: 0.0072 - val_accuracy: 0.9967 - lr: 0.0010
Epoch 5/6
22597/22597 [==============================] - 75s 3ms/step - loss: -0.6324 - accuracy: 0.9975 - val_loss: 0.0046 - val_accuracy: 0.9984 - lr: 0.0010
Epoch 6/6
22597/22597 [==============================] - 75s 3ms/step - loss: -0.6168 - accuracy: 0.9977 - val_loss: 0.0047 - val_accuracy: 0.9984 - lr: 0.0010
Training c

In [11]:
import numpy as np

# Evaluate the global model
global_model = create_model()
global_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
global_model.set_weights(avg_weights[0])
_, accuracy = global_model.evaluate(X_test, y_test)
print(f"Global model accuracy: {accuracy * 100:.2f}%")

# Get predictions
y_pred_probs = global_model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).reshape(-1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

43613/43613 [==============================] - 53s 1ms/step - loss: 0.2160 - accuracy: 0.8994
Global model accuracy: 89.94%
43613/43613 [==============================] - 42s 951us/step
Precision: 0.99
Recall: 0.90
F1-score: 0.95


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
